## Application of causal discovery and causal inference methods to climate dynamics problem
##### Here, this notebook shows and explains step by step a causal approach to analyse linkages among ENSO, circulation fields, and summer monsoon precipitation over the Himalayas.

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")

from src.DataProcessingClass import DataProcessingClass
from src.CausalityClass import CausalityClass

#### Preprocessing of climate indices (or time series data): 
#### ENSO - El Niño–Southern Oscillation
#### WC - Walker cell
#### MHC - Monsoon Hadley cell
#### Himalayan precipitation

In [ ]:
# Create a class instance for data preprocessing
dpc = DataProcessingClass()

# Load climate indices from netCDF files.
D = dpc.load_netcdf_files()

#### Plot the location of the WC/HP and MHC indices

In [ ]:
dpc.plot_orography_map()

#### Transfrom precipitation data with logarithmic function

In [ ]:
# Apply log transform
ind_name='precip'
if ind_name in list(D.keys()): dpc.transform_precip(D, ind_name=ind_name, apply_transform=True)

#### Apply ADF and KPSS statistical tests to indices to check for the stationarity 

In [ ]:
# Use the Augmented Dickey-Fuller unit root test and Kwiatkowski-Phillips-Schmidt-Shin test for stationarity
dpc.adf_kpss_tests(D)

#### 1) Based on the result of statitiscal test perform detrending of WC index
#### 2) Remove climatological and seasonal cycles
#### 3) Normalise indices to a common range
#### 4) Select datapoint from climate indices from a specified timeframe of our interest

In [ ]:
# Select index
ind_name = 'wc'

if ind_name in list(D.keys()): dpc.detrend_index(D, ind_name)

# Remove means
dpc.remove_means(D)

# Apply min max normalisation
dpc.min_max_norm(D)

# Select datapoint for May-September from each year for 1940-2022
dpc.select_datapoints(D)


#### Calculate lagged cross-correlations for all pairs of climate indices

In [ ]:
# Choose max lag time
n_lags = -2

dpc.test_pairs_lag_corr(D, n_lags)

#### Build a causal graph using PCMCI algorithm.

In [ ]:
# Create a class instance with climate indices
ca = CausalityClass(D)

# Construct a custom dataframe format used by Tigramite package
dataframe, _ = ca.generate_dataframe(D) 

# Build the causal graph with PCMCI algorithm
sig_causal_parents = ca.new_construct_causal_graph(dataframe)

#### Use linear estimator to compute beta coefficients on the build causal graph

In [ ]:
# Fit the linear estimator
val_matrix = ca.new_linear_estimator(sig_causal_parents, dataframe)

# Get the beta coefficients for the causal graph
ca.get_links_beta_coeffs()

In [ ]:
ca.linear_mediator.get_coefs()

In [ ]:
ca.linear_mediator.get_coeff(i=0, tau=1, j=0)

In [ ]:
ca.linear_mediator.get_coeff(i=1, tau=1, j=1)

#### Plot the causal graph a causal effect network 

In [ ]:
# Plot causal effect network and save it to a png format
ca.new_plot_cens_graph(val_matrix, save=True)

In [ ]:
val_matrix.diagonal(0)

#### Apply bootstrapping to estimate the statisical robustness of the causal links

In [ ]:
import numpy as np
np.round(np.arange(-1.0, 1.1, 0.5),2)